In [ ]:
# Ch02-3 Pandas Memory

In [ ]:
########################## Notes & Updates ##############################################################
# If you are using Docker and your data directory is mapped to "/data" then you can use the commented-out
#   Docker lines below in place of the primary line (which you will comment out when running)
# You will also find other alternative lines or blocks that can be used to avoid potential issues 
#########################################################################################################

In [ ]:
# Libraries 
import numpy as np 
import pandas as pd 

In [ ]:
# Load data
vdata = pd.read_csv("data/2021VAERSDATA.csv.gz", encoding="iso-8859-1", low_memory=False) 
# vdata = pd.read_csv("/data/2021VAERSDATA.csv.gz", encoding="iso-8859-1", low_memory=False) # Docker
vdata.info(memory_usage="deep") 

In [ ]:
# Inspect the size of each column
for name in vdata.columns:
    col_bytes = vdata[name].memory_usage(index=False, deep=True) 
    col_type = vdata[name].dtype
    print(name, col_type, col_bytes // (1024 ** 2)) 

In [ ]:
# Review the Died column
vdata.DIED.memory_usage(index=False, deep=True) 
vdata.DIED.fillna(False).astype(bool).memory_usage(index=False, deep=True) 

In [ ]:
# State column
vdata["STATE"] = vdata.STATE.str.upper() 
states = list(vdata["STATE"].unique()) 
vdata["encoded_state"] = vdata.STATE.apply(lambda state: states.index(state)) 
vdata["encoded_state"] = vdata["encoded_state"].astype(np.uint8) 
vdata["STATE"].memory_usage(index=False, deep=True) 
vdata["encoded_state"].memory_usage(index=False, deep=True) 

In [ ]:
# Apply optimizations while loading the data
states = list(pd.read_csv("vdata_sample.csv.gz",
    converters={"STATE": lambda state: state.upper()}, 
    usecols=["STATE"] 
)["STATE"].unique()) 

In [ ]:
# Skip the symptom_text column
vdata = pd.read_csv("vdata_sample.csv.gz", index_col="VAERS_ID",
    converters={
        "DIED": lambda died: died == "Y", "STATE": lambda state: states.index(state.upper())
    }, usecols=lambda name: name != "SYMPTOM_TEXT")
vdata["STATE"] = vdata["STATE"].astype(np.uint8)
vdata.info(memory_usage="deep") 

In [ ]:
# Note - it is ok to get a dtype warning on the above

In [ ]:
# End of Notebook #